In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-11"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10329,2024-05-11,Eua Nba,16:30,Dallas Mavericks,Oklahoma City Thunder,1.66,2.31,218.5,1.96,1.80,-3.5,2.00,2.80,MJ8SLDug,0.602410,0.432900,0.510204,0.555556,0.035310,0.2,0.6,NaN,NaN,180.890,33.175685,0.183403,201.524,50.911316,0.252632,174.924,214.968,148.20,148.41,0.0,0.0,0.0,0.0,0.231546,0.060179,0.235702,3.07,0.614,1.074919,0.650629,0.9,0.249371,2.23,0.446,2.937220,0.547640,0.6,0.052360
10330,2024-05-11,Eua Nba,21:30,Cleveland Cavaliers,Boston Celtics,3.78,1.30,210.5,1.80,1.95,6.5,2.05,1.08,IsYOCw8E,0.264550,0.769231,0.555556,0.512821,0.033781,1.0,0.4,NaN,NaN,186.054,30.214720,0.162398,162.568,28.681404,0.176427,177.894,161.782,170.66,121.38,0.0,0.0,0.0,0.0,0.690403,0.056569,0.438271,-0.30,-0.060,-46.333333,0.656803,0.6,-0.056803,-1.00,-0.200,-1.500000,0.698769,0.8,0.101231
10331,2024-05-11,Austrália Nbl1 Leste,06:00,Bankstown Bruins,Albury-Wodonga Bandits,2.05,1.70,166.5,1.81,1.89,-2.5,1.95,1.94,ERn0vuX1,0.487805,0.588235,0.552486,0.529101,0.076040,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,147.42,0.0,0.0,0.0,0.0,0.131993,0.030578,0.003636,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10332,2024-05-11,Austrália Nbl1 Leste,06:00,Illawarra Hawks,Hornsby S.,1.81,1.91,168.5,1.88,1.78,-1.5,1.93,2.03,OxARb9IG,0.552486,0.523560,0.531915,0.561798,0.076046,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.038016,0.038640,0.035712,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10333,2024-05-11,Austrália Nbl1 Sul,06:00,NW Tasmania,Melbourne Tigers,1.69,2.06,164.5,1.84,1.86,4.5,1.80,1.82,6B679f7F,0.591716,0.485437,0.543478,0.537634,0.077153,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,145.92,0.00,0.0,0.0,0.0,0.0,0.139536,0.007644,0.007813,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10334,2024-05-11,Austrália Nbl1 Sul,04:00,Geelong F,Sandringham F,1.84,1.88,158.5,1.83,1.83,-1.5,1.96,1.79,xANGc9t4,0.543478,0.531915,0.546448,0.546448,0.075393,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,140.43,0.00,0.0,0.0,0.0,0.0,0.015207,0.000000,0.064111,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10335,2024-05-11,Austrália Nbl1 Central,08:00,Southern Tigers,Norwood Flames,2.48,1.50,176.5,1.80,1.90,1.5,1.85,1.61,rTqkRFve,0.403226,0.666667,0.555556,0.526316,0.069892,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.348223,0.038222,0.098096,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10336,2024-05-11,Austrália Nbl1 Oeste,06:00,Warwick Senators F,Beeliar Boodjar F,1.99,1.74,158.5,1.85,1.85,1.5,1.85,1.63,YyFubJFh,0.502513,0.574713,0.540541,0.540541,0.077225,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.094786,0.000000,0.089404,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10337,2024-05-11,Austrália Nbl1 Oeste,06:30,Lakeside Lightning F,Willetton Tigers F,1.66,2.13,157.5,1.86,1.84,-2.5,1.87,2.13,fi3ldcp5,0.602410,0.469484,0.537634,0.543478,0.071893,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.175377,0.007644,0.091924,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10338,2024-05-11,Bósnia E Herzegovina Prvenstvo Bih,14:00,Borac Banja Luka,Siroki Brijeg,1.99,1.74,149.5,1.88,1.79,1.5,1.85,1.61,0WnxPElf,0.502513,0.574713,0.531915,0.558659,0

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,16:30,Eua Nba,Dallas Mavericks,Oklahoma City Thunder,218.5,1.96,1.0,Over
1,06:00,Austrália Nbl1 Leste,Illawarra Hawks,Hornsby S.,168.5,1.88,1.0,Over
2,06:30,Austrália Nbl1 Oeste,Lakeside Lightning F,Willetton Tigers F,157.5,1.86,1.0,Over
3,07:15,Filipinas Copa Das Filipinas,Ginebra Kings,Magnolia Hotshots,176.5,1.80,1.0,Over
4,08:00,Indonésia Ibl,Dewa United,Pelita Jaya,165.5,1.85,1.0,Over
5,14:40,Israel Superliga,Maccabi Ironi Ramat Gan,Hapoel Holon,164.5,1.87,1.0,Over
6,01:00,Nova Zelândia Nbl,Bay Hawks,Otago Nuggets,191.5,1.86,1.0,Over
7,08:00,Rússia Liga Vtb United,Lokomotiv Kuban,Unics Kazan,158.5,1.83,1.0,Over
8,21:00,Chile Lnb,Puerto Montt,Espanol de Talca,156.5,1.86,1.0,Over
